<a href="https://colab.research.google.com/github/lelouch711/Projects/blob/main/LunarLanding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing Gymnasium

In [ ]:
!pip install gymnasium
!pip install wheel
!pip install "gymnasium[atari,accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 49 not upgraded.
Need to get 1,116 kB of archives.
After this operation, 5,542 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig4.0 amd64 4.0.2-1ubuntu1 [1,110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig all 4.0.2-1ubuntu1 [5,632 B]
Fetched 1,116 kB in 1s (1,197 kB/s)
Selecting previously unselected package swig4.0.
(Reading database ... 123597 files and directories currently installed.)
Preparing to unpack .../swig4.0_4.0.2-1ubuntu1_amd64.deb ...
Unpacking swig4.0 (4.0.2-1ubuntu1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_4.0.2-1ubu

In [ ]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque,namedtuple
import torch.optim as optim

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Building the AI


In [ ]:
class Network(nn.Module):
    def __init__(self, state_size, action_size,seed=42):
        super(Network, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size,64)
        self.fc2 = nn.Linear(64,64)
        self.fc3 = nn.Linear(64,action_size)
    def forward(self,state):
        x = self.fc1(state)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        return x

Training AI

In [ ]:
import gymnasium as gym
env = gym.make('LunarLander-v2')
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
number_actions = env.action_space.n
print(state_shape)
print(state_size)
print(number_actions)

(8,)
8
4


In [ ]:
learning_rate = 5e-4
minibatch_size = 100
discount_factor = 0.99
replay_buffer_size = int(1e5)
interpolation_parameter = 1e-3


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
class ExperienceReplay(object):
  def __init__(self,capacity):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.capacity = capacity
    self.memory = []
  def push(self,event):
    self.memory.append(event)
    if(len(self.memory)>self.capacity):
      del self.memory[0]
  def sample(self,batch_size):
    events =  random.sample(self.memory,k =batch_size)
    states = torch.from_numpy(np.vstack([e[0] for e in events if e is not None])).float().to(self.device)
    actions = torch.from_numpy(np.vstack([e[1] for e in events if e is not None])).long().to(self.device)
    rewards = torch.from_numpy(np.vstack([e[2] for e in events if e is not None])).float().to(self.device)
    next_states = torch.from_numpy(np.vstack([e[3] for e in events if e is not None])).float().to(self.device)
    dones = torch.from_numpy(np.vstack([e[4] for e in events if e is not None]).astype(np.uint8)).float().to(self.device)
    return states,next_states,actions,rewards,dones

In [ ]:
class Agent():
  def __init__(self,state_size,action_size):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.state_size = state_size
    self.action_size = action_size
    self.local_qnetwork = Network(state_size,action_size).to(self.device)
    self.target_qnetwork = Network(state_size,action_size).to(self.device)
    self.optimiser = optim.Adam(self.local_qnetwork.parameters(),lr = learning_rate)
    self.memory = ExperienceReplay(replay_buffer_size)
    self.t_step = 0

  def step(self,state,action,reward,next_state,done):
    self.memory.push((state,action,reward,next_state,done))
    self.t_step = (self.t_step+1)%4
    if(self.t_step == 0):
      if(len(self.memory.memory)>minibatch_size):
         experiences = self.memory.sample(100)
         self.learn(experiences,discount_factor)
  def act(self,state,epsilon = 0.):
    state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
    self.local_qnetwork.eval()
    with torch.no_grad():
      action_values = self.local_qnetwork(state)
      self.local_qnetwork.train()
      if random.random()>epsilon:
        return np.argmax(action_values.cpu().data.numpy)
      else:
        return np.random.choice(np.arange(self.action_size))

  def learn(self,experiences,discount_factor):
    states,next_states,actions,rewards,dones = experiences
    next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
    q_targets = rewards + (discount_factor*next_q_targets*(1-dones))
    q_expected = self.local_qnetwork(states).gather(1,actions)
    loss = F.mse_loss(q_targets,q_expected)
    self.optimiser.zero_grad()
    loss.backward()
    self.optimiser.step()
    self.soft_update(self.local_qnetwork,self.target_qnetwork,interpolation_parameter)

  def soft_update(seld,local_model,target_model,interpolation_parameter):
    for target_param,local_param in zip(target_model.parameters(),local_model.parameters()):
      target_param.data.copy_(interpolation_parameter*local_param.data+(1.0-interpolation_parameter)*target_param.data)



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
agent = Agent(state_size,number_actions)

In [ ]:
number_episodes = 2000
maximum_nummber_timesteps_per_episode = 1000
epsilon_starting = 1.0
epsilon_end = 0.01
epsilon_decay = 0.995
epsilon = epsilon_starting
scores_on_100_episodes = deque(maxlen=100)

for episodes in range(1,number_episodes):
  state,_ = env.reset()
  score = 0
  for t in range(maximum_nummber_timesteps_per_episode):
    action = agent.act(state,epsilon)
    next_state,reward,done,_,_ = env.step(action)
    agent.step(state,action,reward,next_state,done)
    state = next_state
    score += reward
    if done:
      break
    scores_on_100_episodes.append(score)
    epsilon = max(epsilon_end,epsilon*epsilon_decay)
    print('\r Episode {}\t Average Score {:.2f}'.format(episodes,np.mean(scores_on_100_episodes)),end = "")
    if episodes%100 == 0:
      print(' Episode {}\t Average Score {:.2f}'.format(episodes,np.mean(scores_on_100_episodes)))
    if np.mean(scores_on_100_episodes)>= 200.0:
      print('Environment Solved in {:d} episodes\t Average Score {:.2f}'.format(episodes,np.mean(scores_on_100_episodes)))
      torch.save(agent.local_qnetwork.state_dict(),'checkpoint.pth')
      break

 Episode 100	 Average Score -39.71 Episode 100	 Average Score -39.71
 Episode 100	 Average Score -39.24 Episode 100	 Average Score -39.24
 Episode 100	 Average Score -38.74 Episode 100	 Average Score -38.74
 Episode 100	 Average Score -38.21 Episode 100	 Average Score -38.21
 Episode 100	 Average Score -37.66 Episode 100	 Average Score -37.66
 Episode 100	 Average Score -37.08 Episode 100	 Average Score -37.08
 Episode 100	 Average Score -36.47 Episode 100	 Average Score -36.47
 Episode 100	 Average Score -35.83 Episode 100	 Average Score -35.83
 Episode 100	 Average Score -35.17 Episode 100	 Average Score -35.17
 Episode 100	 Average Score -34.49 Episode 100	 Average Score -34.49
 Episode 100	 Average Score -33.78 Episode 100	 Average Score -33.78
 Episode 100	 Average Score -33.06 Episode 100	 Average Score -33.06
 Episode 100	 Average Score -32.33 Episode 100	 Average Score -32.33
 Episode 100	 Average Score -31.61 Episode 100	 Average Score -31.61
 Episode 100	 Average Score -30.89

In [ ]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gym.wrappers.monitoring.video_recorder import VideoRecorder

def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'LunarLander-v2')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()